# PARETO - Strategic Model Demo
The purpose of this Jupyter notebook is to provide a demonstration of PARETO's strategic model. This demo will show how to use PARETO's Python coding interface. If you prefer a graphical interface, you can download the PARETO GUI [here](https://www.project-pareto.org/software/).

## Introduction
This demo is based on PARETO's strategic toy case study. Relevant links:
- [Strategic model documentation](https://pareto.readthedocs.io/en/latest/model_library/strategic_water_management/index.html)
- [Documentation of PARETO case studies](https://pareto.readthedocs.io/en/latest/case_studies/index.html)

The strategic toy case study features a very small produced water network. This network is smaller than most realistic produced water networks, but the small size of this example makes it useful for testing, debugging, demonstrations, etc. Below is a schematic image of the strategic toy network:

![Strategic toy case study network](strategic_toy_network.png)

Please note that the strategic toy case study data is arbitrary, but meant to be representative of a real produced water network. We will now proceed with demonstrating all of the steps that are needed to set up and solve an instance of PARETO's strategic model.

The rest of this demonstration will proceed in six steps:
1. [Import needed files and libraries](#import)
2. [Set up the set list and parameter list](#set_list_and_parameter_list)
3. [Load data into Python](#load_data)
4. [Build the Pyomo model](#build)
5. [Solve the model](#solve)
6. [Analyze results](#analyze)

Note that PARETO includes a file called `run_strategic_model.py` which is a ready-to-go script that can be used to run a PARETO case study with default options. The steps which are described in this notebook are essentially the same as those that are used in `run_strategic_model.py`.

## <a id='import'></a>Step 1: Import needed files and libraries
To begin, ensure that you are working in a development environment where PARETO is installed. For example, if you followed the PARETO [installation instructions](https://pareto.readthedocs.io/en/latest/getting_started/index.html) to create a conda environment called `pareto-env`, then make sure that environment is active by running `conda activate pareto-env`.

The first thing we do in Python is import all necessary modules and functions:

In [ ]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2023 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

from pareto.strategic_water_management.strategic_produced_water_optimization import (
    WaterQuality,
    RemovalEfficiencyMethod,
    create_model,
    Objectives,
    solve_model,
    PipelineCost,
    PipelineCapacity,
)
from pareto.utilities.get_data import get_data
from pareto.utilities.results import (
    plot_bars,
    plot_sankey,
    generate_report,
    PrintValues,
    OutputUnits,
    is_feasible,
    nostdout,
)
from importlib import resources

## <a id='set_list_and_parameter_list'></a>Step 2: Set up the set list and parameter list
The data defining the produced water network is stored in an Excel workbook (often referred to as the "input file"). Some of the tabs in the input file define index sets for the model, whereas other tabs contain the parameter data that will used to build the model.

A rather large amount of data must be provided by the user. We will not attempt to summarize all of the data input requirements here, but we refer the reader to the [strategic model documentation](https://pareto.readthedocs.io/en/latest/model_library/strategic_water_management/index.html) to learn more. You can also download several example input files from GitHub [here](https://github.com/project-pareto/project-pareto/tree/main/pareto/case_studies). Note that `strategic_toy_case_study.xlsx` is the input file which will be used in this demo.

In [ ]:
# Each entry in set_list corresponds to a tab in the Excel input file that
# defines an index set.
set_list = [
    "ProductionPads", "CompletionsPads", "SWDSites", "FreshwaterSources", "StorageSites",
    "TreatmentSites", "ReuseOptions", "NetworkNodes", "PipelineDiameters", "StorageCapacities",
    "InjectionCapacities", "TreatmentCapacities", "TreatmentTechnologies",
]
# Each entry in parameter_list also corresponds to a tab in the Excel input
# file, but these tabs have parameter data.
parameter_list = [
    "Units", "PNA", "CNA", "CCA", "NNA", "NCA", "NKA", "NRA", "NSA", "FCA", "RCA", "RNA",
    "RSA", "SCA", "SNA", "PCT", "PKT", "FCT", "CST", "CCT", "CKT", "CompletionsPadOutsideSystem",
    "DesalinationTechnologies", "DesalinationSites", "TruckingTime", "CompletionsDemand",
    "PadRates", "FlowbackRates", "NodeCapacities", "InitialPipelineCapacity",
    "InitialDisposalCapacity", "InitialTreatmentCapacity", "FreshwaterSourcingAvailability",
    "PadOffloadingCapacity", "CompletionsPadStorage", "DisposalOperationalCost",
    "TreatmentOperationalCost", "ReuseOperationalCost", "PipelineOperationalCost",
    "FreshSourcingCost", "TruckingHourlyCost", "PipelineDiameterValues",
    "DisposalCapacityIncrements", "InitialStorageCapacity", "StorageCapacityIncrements",
    "TreatmentCapacityIncrements", "TreatmentEfficiency", "RemovalEfficiency",
    "DisposalExpansionCost", "StorageExpansionCost", "TreatmentExpansionCost",
    "PipelineCapexDistanceBased", "PipelineCapexCapacityBased", "PipelineCapacityIncrements",
    "PipelineExpansionDistance", "Hydraulics", "Economics", "PadWaterQuality",
    "StorageInitialWaterQuality", "PadStorageInitialWaterQuality", "DisposalOperatingCapacity",
]

## <a id='load_data'></a>Step 3: Load data into Python
PARETO's `get_data` utility function ([documentation](https://pareto.readthedocs.io/en/latest/utilities/Get_Data.html#get-data-get-data)) called below reads the data from the input file into the `df_sets` and `df_parameters` variables. The variables `df_sets` and `df_parameters` which are returned are dictionaries where the keys are the strings from `set_list` and `parameter_list`, respectively, and the values are either dictionaries or Pandas dataframe objects containing the data read from the corresponding workbook tab.

In [ ]:
# Load data from Excel input file into Python
with resources.path(
    "pareto.case_studies",
    "strategic_toy_case_study.xlsx",
) as fpath:
    [df_sets, df_parameters] = get_data(fpath, set_list, parameter_list)

### Step 3.1 (optional): Display input data
A simple way to view the input data is to print members from `df_sets` and `df_parameters`:

In [ ]:
print("------- Production pads -------")
print(type(df_sets["ProductionPads"]))
print(df_sets["ProductionPads"])

print("\n------- Completions demand -------")
print(type(df_parameters["CompletionsDemand"]))
print(df_parameters["CompletionsDemand"])

print("\n------- Desalination technologies -------")
print(type(df_parameters["DesalinationTechnologies"]))
print(df_parameters["DesalinationTechnologies"])

PARETO's `plot_bars` function ([documentation](https://pareto.readthedocs.io/en/latest/utilities/Results.html#plot-bars)) can be used to create bar charts. If the plotted variable is indexed by time, then a dynamic, animated chart is created:

In [ ]:
input_data = {"pareto_var": df_parameters["PadRates"],
              "labels": [("Production pad", "Time", "Production forecast (bbl/day)")],
             }
args = {"plot_title": "Production forecast",
        "output_file": "demo_bar.html",
        "print_data": False,
        "jupyter_notebook": True,  # setting this option to True causes the bar chart to appear in the Jupyter notebook
       }
plot_bars(input_data, args)

## <a id='build'></a>Step 4: Build the Pyomo model
PARETO's `create_model` function called below uses the [Pyomo](http://www.pyomo.org/) modeling language to build a mathematical model of the produced water network. There are five different settings which can be specified for building the model:

| Setting | Possible values (default in *italics*) |
| :- | :- |
| `objective` | *`Objectives.cost`* - minimize the total annualized cost of produced water management over the decision horizon<br><br>`Objectives.reuse` - maximuze the amount of produced water that is reused over the decision horizon |
| `pipeline_cost` | *`PipelineCost.capacity_based`* - use pipeline capacities and rate in [currency/volume] to calculate pipeline CAPEX costs<br><br>`PipelineCost.distance_based` - use pipeline distances and rate in [currency/(diameter-distance)] to calculate pipeline CAPEX costs |
| `pipeline_capacity` | *`PipelineCapacity.input`* - flow capacity for each pipe diameter is provided by the user<br><br>`PipelineCapacity.calculate` - flow capacity for each pipe diameter is calculated based on the diameter and provided pipe hydraulics data |
| `node_capacity` | *`True`* - include upper bound on network node flow capacity <br><br>`False` - exclude upper bound on network node flow capacity |
| `water_quality` | *`WaterQuality.post_process`* - calculate water quality throughout the network post-optimization<br><br>`WaterQuality.false` - exclude any water quality calculations from the model<br><br>`WaterQuality.discrete` - discretize the water quality variables to include in the optimization model |
| `removal_efficiency_method` | `RemovalEfficiencyMethod.load_based` - treatment component removal efficiency is based on the total load of quality components<br><br>*`RemovalEfficiencyMethod.concentration_based`* - treatment component removal efficiency is based on concentration reduction |

In [ ]:
# Create Pyomo optimization model representing the produced water network
strategic_model = create_model(
    df_sets,
    df_parameters,
    default={
        "objective": Objectives.cost,
        "pipeline_cost": PipelineCost.distance_based,
        "pipeline_capacity": PipelineCapacity.input,
        "node_capacity": True,
        "water_quality": WaterQuality.false,
        "removal_efficiency_method": RemovalEfficiencyMethod.concentration_based,
    },
)

## <a id='solve'></a>Step 5: Solve the model
PARETO's `solve_model` function called below solves the model with the provided options. There are seven different options which can be passed:

| Option | Description | Default value |
| :- | :- | :- |
| `solver` | Either a string with solver name or a tuple of strings with several solvers to try and load in order. PARETO currently supports the Gurobi (commercial) and CBC (free) solvers, but it might be possible to use other MILP solvers as well. | `("gurobi_direct", "gurobi", "cbc")` |
| `deactivate_slacks` | `True` to deactivate slack variables, `False` to use slack variables. | `True` |
| `scale_model` | `True` to apply scaling to the model, `False` to not apply scaling. | `False` |
| `scaling_factor` | Scaling factor to apply to the model (only relevant if `scale_model` is `True``). | 1000000 |
| `running_time` | Maximum solver running time in seconds. | 60 |
| `gap` | Solver gap. | 0 |
| `gurobi_numeric_focus` | The `NumericFocus` parameter to pass to the Gurobi solver. This parameter can be 1, 2, or 3, and per Gurobi, "settings 1-3 increasingly shift the focus towards more care in numerical computations, which can impact performance." This option is ignored if a solver other than Gurobi is used. | 1 |

In [ ]:
# Solve Pyomo model with specified options
options = {
    "solver": "cbc",  # "gurobi" is another solver option if you have it installed
    "deactivate_slacks": True,
    "scale_model": False,
    "scaling_factor": 1000000,
    "running_time": 300,
    "gap": 0,
    "gurobi_numeric_focus": 1,
}
results = solve_model(model=strategic_model, options=options)

PARETO's `is_feasible` utility function can be called after solving the model to check whether or not the solution returned by the solver violates any constraints:

In [ ]:
# Check feasibility of the solved model.
with nostdout():
    feasibility_status = is_feasible(strategic_model)
if not feasibility_status:
    print("Model results are not feasible and should not be trusted")
else:
    print("Model results validated and found to pass feasibility tests")

## <a id='analyze'></a>Step 6: Analyze results
PARETO's `generate_report` function ([documentation](https://pareto.readthedocs.io/en/latest/utilities/Results.html#generate-report)) can be used to export the optimization reports to an Excel report. The first argument to the function is the model itself, and then there are four optional keyword arguments:

| Option | Description | Default value |
| :- | :- | :- |
| `results_obj` | The object returned by the solver. If provided, this object is used to extract data from the solver to include in the report. | `None` |
| `is_print` | Determines how many results to print to the console, but has no effect on what is included in the Excel report. There are four possible values that may be passed:<br><br>- `None`<br><br>- `PrintValues.detailed`<br><br>- `PrintValues.nominal`<br><br>- `PrintValues.essential`<br><br>Note that it will be a rare case where it is useful or desirable to pick something other than `None` or `PrintValues.essential` for this option. | `None` |
| `output_units` | Determines whether the units used in the report are the units provided by the user or the internal units used by the model. Specifically, the values that may be passed are:<br><br>- `OutputUnits.unscaled_model_units`<br><br>- `OutputUnits.user_units` | `OutputUnits.user_units` |
| `fname` | The file name for the generated Excel report. | `"PARETO_report.xlsx"` |

In [ ]:
# Generate report with results in Excel
[model, results_dict] = generate_report(
    strategic_model,
    results_obj=results,
    is_print=PrintValues.essential,
    output_units=OutputUnits.user_units,
    fname="strategic_optimization_results.xlsx",
)

The `get_data` function can also be used to load data from a previously generated PARETO results report back into Python:

In [ ]:
# Read data from the PARETO results report
tabs_to_read = ["v_F_Overview", "vb_y_overview", "v_F_Sourced"]
fname = "strategic_optimization_results.xlsx"
_, parameters_report = get_data(fname, [], tabs_to_read)

print("------- Overview -------")
print(parameters_report["v_F_Overview"])  # Note that this is the same data that was printed above by the generate_report function
print("\n------- Infrastructure buildout overview -------")
print(parameters_report["vb_y_overview"])
print("\n------- Sourced water -------")
print(parameters_report["v_F_Sourced"])

### Step 6.1 (optional): Generate Sankey diagrams
PARETO's `plot_sankey` function ([documentation](https://pareto.readthedocs.io/en/latest/utilities/Results.html#generate-sankey)) can be used to generate Sankey diagrams of the output:

In [ ]:
input_data = {"pareto_var": results_dict["v_F_Piped_dict"], 
              "time_period": "T01",
             }
args = {"plot_title": "Piped water - T01",
        "output_file": "demo_sankey_T01.html",
        "jupyter_notebook": True,  # setting this option to True causes the Sankey diagram(s) to appear in the Jupyter notebook
       }
plot_sankey(input_data, args)

Note that the Sankey diagram is interactive - try hovering your mouse over different elements of the diagram, and clicking and dragging!

We can pass a list of multiple times for the `time_period` entry in the `input_data` argument to get a Sankey diagram showing the cumulative flows over multiple time periods:

In [ ]:
input_data = {"pareto_var": results_dict["v_F_Piped_dict"], 
              "time_period": ["T01", "T02"],
             }
args = {"plot_title": "Piped water - T01 and T02",
        "output_file": "demo_sankey_T01_T02.html",
        "jupyter_notebook": True,
       }
plot_sankey(input_data, args)

If we omit the `time_period` entry entirely, then the flows shown in the Sankey diagram are cumulative for the entire time horizon:

In [ ]:
input_data = {"pareto_var": results_dict["v_F_Piped_dict"]}
args = {"plot_title": "Piped water - all times",
        "output_file": "demo_sankey_all_times.html",
        "jupyter_notebook": True,
       }
plot_sankey(input_data, args)

By passing a `sections` entry in the `input_data` argument, we can create separate Sankey diagrams for different subsections of the overall network: 

In [ ]:
input_data = {"pareto_var": results_dict["v_F_Piped_dict"], 
              "sections": {"Section1": ["N01", "N02", "N03", "N04"],
                           "Section2": ["N05", "N06", "N07", "N08", "N09"]},
              "time_period": "T01",
             }
args = {"plot_title": "Piped water - T01",
        "output_file": "demo_sankey_T01_sections.html",
        "jupyter_notebook": True,
       }
plot_sankey(input_data, args)

## Conclusion
We hope this demonstration of PARETO's strategic model has been helpful and informative. If you have any questions, comments, or feedback for the PARETO team, please do not hesitate to [contact us](https://www.project-pareto.org/team/contact-us/).